# Cholec80 Processing
 - Find center of laparoscopic images via RANSAC under linear regression model
 - Find maximum rectangle crop of given ratio around center
 - Export found transforms as yaml file

## Note
Note, however, that it would be easier, and potentially more stable, to add the circle as part of the augmentation pipeline. Therefore, following transforms should be considered under utils.transforms

 - Constant circle, no circle, moving circle
 - Noisy boundary
 - Rectangle, moving rectangle, rescaling

In [2]:
import os
import cv2
import numpy as np
import yaml
import sys

sys.path.append('../')

from utils.io import scan2df, save_yaml, load_yaml
from utils import endoscopy

servers = load_yaml('configs/servers.yml')
server = 'local'

In [2]:
prefix = servers[server]['database']['location']

# scan through data
folder = os.path.join(prefix, 'cholec80/videos')

df = scan2df(folder, postfix='.mp4')

# create object to save found transforms
database =  {
    'databases': [{
        'name': 'cholec80',
        'prefix': 'cholec80',
        'test': False,
        'transforms': [],
        'videos': {
            'files': [],
            'prefix': 'videos'
}}]}

df

,folder,file
0,/media/martin/Samsung_T5/data/endoscopic_data/...,video36.mp4
1,/media/martin/Samsung_T5/data/endoscopic_data/...,video37.mp4
2,/media/martin/Samsung_T5/data/endoscopic_data/...,video52.mp4
3,/media/martin/Samsung_T5/data/endoscopic_data/...,video38.mp4
4,/media/martin/Samsung_T5/data/endoscopic_data/...,video53.mp4
...,...,...
75,/media/martin/Samsung_T5/data/endoscopic_data/...,video33.mp4
76,/media/martin/Samsung_T5/data/endoscopic_data/...,video34.mp4
77,/media/martin/Samsung_T5/data/endoscopic_data/...,video50.mp4
78,/media/martin/Samsung_T5/data/endoscopic_data/...,video51.mp4


In [4]:
# find center, radius and rectangle crop
visualize = True
stable_steady_count = 50  # center is defined stable after 50 consecutive images have the same center
stable_zoom_count = 1000  # zoom is considered stable, after 1000 consecutive images are zoomed

steady_tol = 1.0
zoom_tol = 0.99

# offset crop to remove black borders
offset = 5

# output
out_prefix = os.path.join(os.getcwd(), 'configs')
out_file = 'cholec80_transforms.yml'

dsize = [640, 480] # opencv convention for resize output


for idx, row in df.iterrows():
    ib = endoscopy.ImageBuffer(buffer_size=50)

    vc = cv2.VideoCapture(os.path.join(row.folder, row.file))

    print('Processing file: {} with index: {}'.format(row.file, idx))

    # steady center check
    steady_count = 0
    prev_center, prev_radius = np.array([0, 0], dtype=np.int), None

    # zoom check
    zoom_count = 0

    # found flags
    center_found = False
    zoom = False

    while vc.isOpened():

        _, img = vc.read()
        if img is None:
            break

        img = img[offset:-offset, offset:-offset]
        ib.appendBuffer(img)

        avg_binary = ib.binaryAvg(th=20)

        center, radius = endoscopy.ransacBoundaryCircle(avg_binary, th=10, fit='numeric', n_pts=100, n_iter=10)
        top_left, shape = endoscopy.boundaryRectangle(avg_binary, th=200)

        # check for circle fit
        if radius is not None:
            # find max inner rectangle
            inner_top_left, inner_shape = endoscopy.maxRectangleInCircle(img.shape, center, radius)
            inner_top_left, inner_shape = inner_top_left.astype(np.int), tuple(map(int, inner_shape))
    
            center, radius = center.astype(np.int), int(radius)
            top_left, shape = top_left.astype(np.int), tuple(map(int, shape))

            if steady_count == 0:
                prev_center, prev_radius = center, radius
                steady_count += 1
            else:
                if np.isclose(prev_center, center, atol=steady_tol).all():
                    prev_center, prev_radius = center, radius
                    steady_count += 1

                    if steady_count >= stable_steady_count + 1:
                        center_found = True
                        break
                else:
                    prev_center, prev_radius = np.array([0, 0], dtype=np.int), None
                    steady_count = 0

            if visualize:
                cv2.circle(img, (center[1], center[0]), radius, (0, 255, 255))
                cv2.circle(img, (center[1], center[0]), 2, (0, 255, 255))
                cv2.rectangle(img, (top_left[1], top_left[0]), (top_left[1]+shape[1], top_left[0]+shape[0]), (255, 0, 255))
                cv2.rectangle(
                    img, 
                    (inner_top_left[1], inner_top_left[0]), 
                    (inner_top_left[1]+inner_shape[1], inner_top_left[0]+inner_shape[0]), 
                    (255, 255, 0)
                )

        # check for zoom
        zoomed, confidence = endoscopy.isZoomed(avg_binary, th=zoom_tol)
        if zoomed:
            zoom_count += 1
        else:
            zoom_count = 0
        if zoom_count > stable_zoom_count:
            zoom = True
            break

        if visualize:
            cv2.imshow('avg_binary', avg_binary)
            cv2.imshow('img', img)
            cv2.waitKey(1)
        
        print('\rSteady count: {}, zoom count: {}'.format(steady_count, zoom_count), end='')

    vc.release() 
    print('\nCenter found: {}, zoom: {}'.format(center_found, zoom))
    print('Center found at: ', center, '\n')

    # save results, remember to add offset again
    if center_found and not zoom:
        database['databases'][0]['transforms'].append(
            [{'Crop': {'top_left_corner': [inner_top_left.item(0) + offset, inner_top_left.item(1) + offset], 'shape': [inner_shape[0], inner_shape[1]]}}, {'Resize': {'dsize': dsize}}]
        )
        database['databases'][0]['videos']['files'].append(row.file)

    if visualize:
        if radius is not None:
            cv2.circle(img, (center[1], center[0]), radius, (0, 255, 255))
            cv2.circle(img, (center[1], center[0]), 2, (0, 255, 255))
            cv2.rectangle(img, (top_left[1], top_left[0]), (top_left[1]+shape[1], top_left[0]+shape[0]), (255, 0, 255))
            cv2.rectangle(
                img, 
                (inner_top_left[1], inner_top_left[0]), 
                (inner_top_left[1]+inner_shape[1], inner_top_left[0]+inner_shape[0]), 
                (255, 255, 0)
            )

        cv2.imshow('avg_binary', avg_binary)
        cv2.imshow('img', img)
        cv2.waitKey()

# # save resulting yaml file
# save_yaml(os.path.join(out_prefix, out_file), database)



Processing file: video36.mp4 with index: 0
Steady count: 50, zoom count: 0
Center found: True, zoom: False
Center found at:  [226 438] 

Processing file: video37.mp4 with index: 1
Steady count: 50, zoom count: 0
Center found: True, zoom: False
Center found at:  [240 436] 

Processing file: video52.mp4 with index: 2
Steady count: 50, zoom count: 0
Center found: True, zoom: False
Center found at:  [257 434] 

Processing file: video38.mp4 with index: 3
Steady count: 50, zoom count: 0
Center found: True, zoom: False
Center found at:  [253 441] 

Processing file: video53.mp4 with index: 4
Steady count: 50, zoom count: 0
Center found: True, zoom: False
Center found at:  [216 427] 

Processing file: video54.mp4 with index: 5
Steady count: 50, zoom count: 0
Center found: True, zoom: False
Center found at:  [249 436] 

Processing file: video39.mp4 with index: 6
Steady count: 50, zoom count: 0
Center found: True, zoom: False
Center found at:  [235 398] 

Processing file: video55.mp4 with index: 